## Importation des biblioteque

In [2]:
import numpy as np
from sqlalchemy import create_engine
import glob as g 
import pandas as pd 

In [3]:
# Si c'est pas installer 
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


  
#### configuration du moteur pour se connecter à une base de données Mysql appelée "ETL_DATABASE" sur localhost (votre propre machine) au port 3306, en utilisant le nom d'utilisateur "root" et le mot de passe "".

In [29]:
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Accéder aux variables d'environnement chargées
user_name = os.getenv("USER_NAME")
password = os.getenv("PASSWOR")



In [34]:
engine = create_engine("mysql+mysqlconnector://"+str(user_name)+":@localhost:3306/elt_database")

### Création de classe pour le processus ETL

In [35]:
class ETLProcess : 
    def __init__(self):
        pass
    
    # Extraction
    
    def load_csv(self):
        dico = {}
        fichiers = g.glob("*.csv")
        for file in fichiers :
            dico[file] = pd.read_csv(file)
        # print(dico)
        return dico
    
    def load_json(self) :
        dico = {}
        
        fichiers = g.glob("*.json")
        for file in fichiers :
            dico[file] = pd.read_json(file)
        return dico
    
    def load_sql(self) :
        data = pd.read_sql('SELECT * FROM table_source', engine)
        return data
    
    # Transformation
    
    def transform(self) :
        data_csv = self.load_csv().get("data.csv").drop(columns = ["id"], axis=1)
        data_json = self.load_json().get("data.json").drop(columns=["id"], axis=1)
        data_sql = self.load_sql().drop(columns=["id"])
        data = pd.concat([data_csv, data_json, data_sql], axis=0, ignore_index=True)
        data["majeur"] = data['age'] >= 18
        return data
    
    def save(self) :
        data = self.transform()
        data.to_sql("table_final1", engine)
        print("!!!! Table crée")
    
        

In [36]:
if __name__ == "__main__" :
    etl = ETLProcess()
    
    etl.save()
    

!!!! Table crée
